In [1]:
pip install python-pptx beautifulsoup4 lxml


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pptx import Presentation
from bs4 import BeautifulSoup
import os
import re
import sqlite3
import json

conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS powerpoint
                (id INTEGER PRIMARY KEY, drill LIST, years TEXT, age TEXT, repayment TEXT, stipend TEXT, estimated TEXT, program TEXT)''')


In [3]:
# Function to extract text from PowerPoint files
def extract_text_from_pptx(file_path):
    prs = Presentation(file_path)
    text = ""
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += shape.text
    return text

In [4]:
#Not Used but if you wanted to search for a word and the surrounding words
def scrape_words_of_interest(text, words_of_interest):
    soup = BeautifulSoup(text, 'lxml')
    words = soup.get_text().split()
    words_and_next_7 = []
    for i, word in enumerate(words):
        if word in words_of_interest:
            # Extract the next 7 words
            next_7_words = " ".join(words[i+1:i+8])
            words_and_next_7.append((word, next_7_words))
    return words_and_next_7

# Function to extract numbers from text
def extract_numbers(text):
    numbers = re.findall(r'\b\d+\b', text)
    return numbers

# Function to find numbers and their surrounding words
def find_numbers_with_context(text):
    pattern = r'(\b\w+\b\s+){0,5}\b(\d+)\b(\s+\b\w+\b){0,5}'
    matches = re.findall(pattern, text)
    results = []
    for match in matches:
        # Combine the surrounding words with the number
        context = " ".join(match)
        results.append(context)
    return results
list_num = []
# Iterate through PowerPoint files in the directory
for filename in os.listdir(pptx_directory):
    if filename.endswith(".pptx"):
        file_path = os.path.join(pptx_directory, filename)
        text = extract_text_from_pptx(file_path)
        numbers_with_context = find_numbers_with_context(text)
        if numbers_with_context:
            #print(f"Numbers found in {filename}:")
            for context in numbers_with_context:
                #print(context)
                list_num.append(context)
                info = dict_list(list_num)
                sql_add(info)
        else:
            numbers = re.findall(r'\b\d+\b', text)
            if numbers:
                #print(f"Numbers found in {filename}:")
                for number in numbers:
                    #print(number)
                    list_num.append(number)

NameError: name 'pptx_directory' is not defined

In [ ]:
# Directory containing PowerPoint files
pptx_directory = "/Users/annivoigt/Documents/python/powerpoints"
# Words of interest
words_of_interest = ["navy", "carrier", "person"]

In [ ]:
# Iterate through PowerPoint files in the directory
for filename in os.listdir(pptx_directory):
    if filename.endswith(".pptx"):
        file_path = os.path.join(pptx_directory, filename)
        text = extract_text_from_pptx(file_path)
        words_and_next_7 = scrape_words_of_interest(text, words_of_interest)
        for word, next_7 in words_and_next_7:
            print(f"Word of interest '{word}' found in {filename}. Following 7 words: {next_7}")


In [6]:
#These are words of interest that could be changed to find words that exist in Daily briefs
def dict_list(list_num):
    my_information = {"drill":[],"years":[], "age":[], "repayment":[], "stipend":[], "estimated":[],'program':[]}
    for i in list_num:
        if 'drill' in i:
            num = re.findall(r'\d+', i)
            my_information["drill"].append(num[0])
        if 'years' in i:
            num = re.findall(r'\d+', i)
            my_information["years"].append(num[0])
        if 'age' in i:
            num = re.findall(r'\d+', i)
            my_information["age"].append(num[0])
        if 'repayment' in i:
            num = re.findall(r'\d+', i)
            my_information["repayment"].append(num[0])
        if 'stipend' in i:
            num = re.findall(r'\d+', i)
            my_information["stipend"].append(num[0])
        if 'estimated' in i:
            num = re.findall(r'\d+', i)
            my_information["estimated"].append(num[0])
        if 'program' in i:
            num = re.findall(r'\d+', i)
            my_information["program"].append(num[0])
    return(my_information)


def sql_add(my_information):
    age_json = json.dumps(my_information['age'])
    drill_json = json.dumps(my_information['drill'])
    years_json = json.dumps(my_information['years'])
    repayment_json  = json.dumps(my_information['repayment'])
    stipend_json = json.dumps(my_information['stipend'])
    estimate_json = json.dumps(my_information['estimated'])
    program_json = json.dumps(my_information['program'])

    cursor.execute('INSERT INTO powerpoint (drill, years, age, repayment, stipend, estimated, program) VALUES (?, ?, ?, ?, ?, ?, ?)',\
            (drill_json, years_json,\
            age_json, repayment_json, \
            stipend_json, estimate_json, program_json))


Below is Table edits for Future use

In [ ]:

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS powerpoint
                  (id INTEGER PRIMARY KEY, drill LIST, years TEXT, age TEXT, repayment TEXT, stipend TEXT, estimated TEXT, program TEXT)''')

# Insert values from dictionary into the table
cursor.execute('INSERT INTO powerpoint (drill, years, age, repayment, stipend, estimated, program) VALUES (?, ?, ?, ?, ?, ?, ?)',\
               (drill_json, years_json,\
                age_json, repayment_json, stipend_json, estimate_json, program_json))

# Commit the transaction
conn.commit()

# Close connection
conn.close() 

In [8]:
#This is how users could connect to the database and look at what exists in the database 
# -- we can create analytic products based off of this information
conn = sqlite3.connect('example.db')
cursor = conn.cursor()



def query_database(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows


# Example query to select all data from the 'person' table
query = 'SELECT age FROM powerpoint'

# Query the database
results = query_database(conn, query)
print(results)

# Close connection
conn.close()

[('["42"]',)]
